# **VC Benchmark**


# **Overview**

This notebook builds a synthetic yet realistic **benchmarking system** used for evaluating venture capital portfolios. It includes four interlinked tables that simulate how institutional asset managers compare fund performance against sectoral or regional indices.

### Summary of Benchmark Tables

| Table Name                        | Description                                                                 | Key Purpose                                  |
|-----------------------------------|-----------------------------------------------------------------------------|----------------------------------------------|
| **BENCHMARK_GENERAL_INFORMATION** | 20 synthetic benchmarks with sector/geography-based names and unique codes  | Define the benchmark universe                |
| **BENCHMARK_CHARACTERISTICS**     | Summary statistics like Median IRR, MOIC, Fund Count, with units and currencies | Add context and comparability to benchmarks  |
| **BENCHMARK_PERFORMANCE**         | Simulated returns (IRR, MOIC, etc.) across multiple time periods            | Enable time-series performance analysis      |
| **BENCHMARK_ACCOUNT_ASSOCIATION** | Links each account (from the ACCOUNTS_MASTER table) to 2–3 benchmarks with ranked relevance | Enables account-vs-benchmark comparisons |

---

## **Table 1: BENCHMARK_GENERAL_INFORMATION**

The `BENCHMARK_GENERAL_INFORMATION` table defines a list of **10 benchmarks**—**3 real‐world indices** and **7 synthetic benchmarks**—that Venture Capital (VC) funds in our simulation can be evaluated against. Each row represents a benchmark commonly used in institutional performance reporting—either a well-known public index or a simulated sector/geography/combination benchmark.

### Composition

- **Real benchmarks (3):**  
  - `SP_500`  — S&P 500 Index  
  - `R2500`   — Russell 2500 Index  
  - `MSCI_WD` — MSCI World Index  

- **Synthetic benchmarks (7):** Generated via random combinations of provider, region, and/or sector.

### Synthetic Generation Logic

We generate **7 unique synthetic** records using one of three patterns:

| Pattern Type | Name Example                                                 | Code Example    |
|--------------|--------------------------------------------------------------|-----------------|
| `geo`        | PitchBook Europe VC Benchmark                                | `PB_EU`         |
| `sector`     | Preqin CleanTech VC Performance Index                        | `PR_CT`         |
| `combo`      | Cambridge Associates North America Healthcare Growth Index   | `CA_NA_HC`      |

### Key Logic and Validation

| Step                       | Description                                                                                                                              |
|----------------------------|------------------------------------------------------------------------------------------------------------------------------------------|
| **1. Provider Prefix**     | Extracts a stable 2-letter code from the provider name: <br>e.g., `"PitchBook"` → `"PB"`                                                   |
| **2. Geography Mapping**   | Maps region names like `"Asia-Pacific"` to short codes like `"AP"`                                                                        |
| **3. Sector Mapping**      | Maps sectors like `"Healthcare"` to codes like `"HC"`                                                                                    |
| **4. Code Construction**   | Combines provider, region, and/or sector codes to form a machine-readable `BENCHMARKCODE`                                                  |
| **5. Real vs Synthetic**   | Lists 3 fixed real benchmarks explicitly; generates 7 synthetic ones programmatically                                                     |
| **6. Uniqueness**          | Appends numeric suffixes (`_2`, `_3`, etc.) if a generated code already exists                                                            |

In [ ]:
import pandas as pd
import numpy as np
import random
import re
from IPython.display import display, HTML

# Ensure reproducibility
random.seed(42)
np.random.seed(42)

def get_provider_prefix(name: str) -> str:
    """
    Data Validation Logic 1:
    Derive a stable 2-letter prefix from a provider name by:
      - Extracting the first two uppercase letters if available (e.g., 'PitchBook' → 'PB')
      - Otherwise falling back to the first two characters uppercased.
    """
    caps = re.findall(r'[A-Z]', name)
    if len(caps) >= 2:
        return ''.join(caps[:2])
    return name[:2].upper()

# Map full geography names to 2-letter codes
REGION_MAP = {
    "U.S.":             "US",
    "North America":    "NA",
    "Global":           "GL",
    "Asia-Pacific":     "AP",
    "Europe":           "EU",
    "Emerging Markets": "EM",
    "Canada":           "CA"
}

# Pools of inputs
PROVIDERS        = ["PitchBook", "Cambridge Associates", "Preqin", "CB Insights", "CB Insights", "Carta"]
GEOGRAPHIES      = list(REGION_MAP.keys())
SECTORS          = ["Tech", "Healthcare", "CleanTech", "AI", "Growth Equity"]

# Explicit sector code map for clarity in codes
SECTOR_CODE_MAP = {
    "Tech":           "TE",
    "Healthcare":     "HC",
    "CleanTech":      "CT",
    "AI":             "AI",
    "Life Sciences":  "LS",
    "Growth Equity":  "GE"
}

# Suffix pools for different naming patterns
GEO_SUFFIXES     = ["Venture Capital Index", "VC Benchmark", "Private Equity Index", "Growth Equity Index", "Private Capital Index"]
SECTOR_SUFFIXES  = ["VC Performance Index", "Venture Capital Index", "Private Equity Index", "Growth Equity Benchmark"]
COMBO_SUFFIXES   = ["Venture Capital Index", "Growth Index", "VC Performance Index"]

BENCHMARK_NAMES = []
BENCHMARK_CODES = []
USED_CODES      = set()

# Add 3 traditional benchmarks
TRADITIONAL = [
    ("SP_500",  "S&P 500 Index"),
    ("R2500",   "Russell 2500 Index"),
    ("MSCI_WD", "MSCI World Index")
]
for code, name in TRADITIONAL:
    USED_CODES.add(code)
    BENCHMARK_CODES.append(code)
    BENCHMARK_NAMES.append(name)

# Generate exactly 7 synthetic benchmark entries
for _ in range(7):  # Change n here if more benchmarks are needed
    provider = random.choice(PROVIDERS)
    pattern  = random.choice(["geo", "sector", "combo"])

    if pattern == "geo":
        region_full = random.choice(GEOGRAPHIES)
        suffix      = random.choice(GEO_SUFFIXES)
        name        = f"{provider} {region_full} {suffix}"
        pfx = get_provider_prefix(provider)
        rfx = REGION_MAP[region_full]
        base_code = f"{pfx}_{rfx}"

    elif pattern == "sector":
        sector_full = random.choice(SECTORS)
        suffix      = random.choice(SECTOR_SUFFIXES)
        name        = f"{provider} {sector_full} {suffix}"
        pfx = get_provider_prefix(provider)
        sfx = SECTOR_CODE_MAP[sector_full]
        base_code = f"{pfx}_{sfx}"

    else:
        region_full = random.choice(GEOGRAPHIES)
        sector_full = random.choice(SECTORS)
        suffix      = random.choice(COMBO_SUFFIXES)
        name        = f"{provider} {region_full} {sector_full} {suffix}"
        pfx = get_provider_prefix(provider)
        rfx = REGION_MAP[region_full]
        sfx = SECTOR_CODE_MAP[sector_full]
        base_code = f"{pfx}_{rfx}_{sfx}"

    # Ensure uniqueness
    code = base_code
    counter = 1
    while code in USED_CODES:
        counter += 1
        code = f"{base_code}_{counter}"
    USED_CODES.add(code)

    BENCHMARK_NAMES.append(name)
    BENCHMARK_CODES.append(code)

# Assemble into a DataFrame with UPPER_SNAKE_CASE column names
df_benchmark_general = pd.DataFrame({
    "BENCHMARK_CODE":  BENCHMARK_CODES,
    "BENCHMARK_NAME":  BENCHMARK_NAMES
})

# Render as HTML table with UPPER_SNAKE_CASE table name
html = df_benchmark_general.to_html(index=False, classes="mytable")
styled = f"""
<style>
  .mytable th {{ text-align: center !important; }}
  .mytable td {{ text-align: left   !important; }}
</style>
{html}
"""

print("BENCHMARK_GENERAL_INFORMATION")
display(HTML(styled))

BENCHMARK_GENERAL_INFORMATION


BENCHMARK_CODE,BENCHMARK_NAME
SP_500,S&P 500 Index
R2500,Russell 2500 Index
MSCI_WD,MSCI World Index
CA_US,Carta U.S. Private Equity Index
CA_NA,Cambridge Associates North America Venture Capital Index
CA_EU_TE,Carta Europe Tech VC Performance Index
CB_US,CB Insights U.S. Venture Capital Index
CA_EU,Cambridge Associates Europe Private Capital Index
PB_NA_GE,PitchBook North America Growth Equity Growth Index
CA_GE,Cambridge Associates Growth Equity Private Equity Index


In [ ]:
# -- Snowflake SQL table creation

# CREATE TABLE BENCHMARK_GENERAL_INFORMATION (
#     BENCHMARK_CODE     VARCHAR(50) PRIMARY KEY,   -- Unique identifier (e.g., PB_EU)
#     BENCHMARK_NAME     VARCHAR(255)               -- Human-readable name (e.g., PitchBook Europe VC Benchmark)
# );

---

## **Table 2: BENCHMARK_CHARACTERISTICS**

The `BENCHMARK_CHARACTERISTICS` table captures key summary statistics and structural characteristics for each venture capital benchmark. It supports comparisons of performance, composition, and reporting within fact sheets. Each row represents one benchmark–statistic combination (e.g., MEDIAN_IRR for a benchmark).

### **Generation Logic**

We dynamically generate one row per benchmark and characteristic defined for each `BENCHMARK_CODE` in the `BENCHMARK_GENERAL_INFORMATION` table. Values are simulated within realistic ranges, with metadata and currency logic inferred from benchmark names and regional mapping.

| Column Name             | Type      | Example           | Notes                                                |
|-------------------------|-----------|-------------------|------------------------------------------------------|
| `BENCHMARK_CODE`        | String    | `PB_US_TE`        | Foreign key from `BENCHMARK_GENERAL_INFORMATION`     |
| `INCEPTION_YEAR`        | Integer   | `2017`            | Random year between 2010 and 2022                    |
| `CURRENCY_CODE`         | String    | `USD`             | Derived by region mapping or default-weighted random |
| `CURRENCY`              | String    | `US Dollar`       | Full currency name                                   |
| `CATEGORY`              | String    | `VC Benchmark`    | Always set as 'VC Benchmark'                         |
| `CATEGORY_NAME`         | String    | `Venture Capital` | Always set as 'Venture Capital'                      |
| `CHARACTERISTIC_NAME`   | String    | `Median IRR`      | Statistic label (e.g., Median IRR, Mean MOIC, etc.)  |
| `STATISTIC_TYPE`        | String    | `Median`          | Underlying type (Median, Count, NumSecurities, etc.) |
| `UNIT`                  | String    | `%`               | Unit for the value (%, #, etc.)                      |
| `CHARACTERISTIC_VALUE`  | Float/Int | `10.5`            | Simulated value, type depends on characteristic      |
| `HISTORY_DATE`          | Date      | `2025-07-30`      | As-of date (current date or snapshot)                |


### **Key Logic and Validation**

| Step                                  | Description                                                                                                                              |
|----------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------|
| **1. Row Generation**                  | Number of rows = #benchmarks × #characteristics (e.g., MEDIAN_IRR, FUND_COUNT, NUM_SECURITIES, etc.)                                     |
| **2. Name-Based Metadata Extraction**  | Currency, region, and sector are inferred from benchmark name if possible, otherwise assigned randomly with controlled weights.          |
| **3. Simulation Logic**                | All values (e.g., IRR, MOIC, COUNT, NUM_SECURITIES) are simulated using business logic and controlled random distributions.              |
| **4. Range Validation**                | Ranges and types enforced for realism and clean data (e.g., IRR 5–25%, FUND_COUNT 40–150, NUM_SECURITIES 20–500, S&P 500 set to 500, etc.) |

In [ ]:
import pandas as pd
import numpy as np
import random
import yfinance as yf
from IPython.display import display, HTML
from datetime import datetime

# Ensure reproducibility
random.seed(42)
np.random.seed(42)

BENCHMARK_CODES = df_benchmark_general["BENCHMARK_CODE"].tolist()
BENCHMARK_NAMES = df_benchmark_general["BENCHMARK_NAME"].tolist()

UNIT_MAP = {
    "Median":     "%",
    "Mean":       "%",
    "Percentile": "%",
    "Count":      "#",
    "StdDevIRR":  "%",
    "NumSecurities": "#"
}

CURRENCY_CODES = ["USD", "EUR", "CAD", "JPY", "GBP"]
CURRENCY_WEIGHTS = [0.80, 0.05, 0.05, 0.05, 0.05]
CURRENCY_NAME_MAP = {
    "USD": "US Dollar", "EUR": "Euro", "CAD": "Canadian Dollar",
    "JPY": "Japanese Yen", "GBP": "British Pound"
}
REGION_CURRENCY_MAP = {
    "Europe": "EUR", "Canada": "CAD", "U.S.": "USD", "North America": "USD",
    "Global": "USD", "Asia-Pacific": "USD", "Emerging Markets": "USD"
}

CHAR_DEFS = [
    {"name": "Median IRR",      "type": "Median"},
    {"name": "Mean MOIC",       "type": "Mean"},
    {"name": "Top Quartile DPI","type": "Percentile"},
    {"name": "Fund Count",      "type": "Count"},
    {"name": "Std Dev IRR",     "type": "StdDevIRR"},
    {"name": "# of Securities", "type": "NumSecurities"}
]

INCEPTION_YEARS = list(range(2010, 2023))
TODAY = datetime.today().strftime('%Y-%m-%d')

# Fixed securities for known indices
FIXED_SECURITIES = {
    "SP_500": 500,
    "R2500": 2500,
    "MSCI_WD": 1600  # Approximate, adjust if needed
}

# --- 1. Define yfinance tickers and a function for index meta ---
INDEX_META = {
    "SP_500":   {"yf": "^GSPC"},
    "R2500":    {"yf": "^R25I"},  # Note: "R25I" is not always present; "IWM" or "^RUT" are common proxies for Russell 2000
    "MSCI_WD":  {"yf": "URTH"},   # MSCI World ETF as proxy; "URTH" trades in USD; for exact, use "MXWO.L" for GBP
}
def get_index_info(bench_code):
    yf_code = INDEX_META[bench_code]["yf"]
    ticker = yf.Ticker(yf_code)
    info = ticker.info

    # Inception year: get first available data or fund inception
    if "firstTradeDateEpochUtc" in info:
        dt = datetime.utcfromtimestamp(info["firstTradeDateEpochUtc"] / 1000)
        year = dt.year
    elif "inceptionDate" in info and info["inceptionDate"]:
        dt = datetime.utcfromtimestamp(info["inceptionDate"])
        year = dt.year
    else:
        year = 2000  # Fallback

    # Currency
    currency_code = info.get("currency", "USD")
    currency_name = CURRENCY_NAME_MAP.get(currency_code, "Unknown")
    return year, currency_code, currency_name

# --- 2. Main Simulation ---
char_records = []
for bench_code, bench_name in zip(BENCHMARK_CODES, BENCHMARK_NAMES):
    if bench_code in INDEX_META:
        # Get from yfinance
        try:
            inception_year, currency_code, currency_name = get_index_info(bench_code)
        except Exception as e:
            # fallback if API fails
            inception_year, currency_code, currency_name = 2000, "USD", "US Dollar"
    else:
        inception_year = random.choice(INCEPTION_YEARS)
        # Region-based currency assignment
        currency_code = None
        for region, code in REGION_CURRENCY_MAP.items():
            if region in bench_name:
                currency_code = code
                break
        if currency_code is None:
            currency_code = random.choices(CURRENCY_CODES, weights=CURRENCY_WEIGHTS, k=1)[0]
        currency_name = CURRENCY_NAME_MAP[currency_code]

    for char in CHAR_DEFS:
        if char["type"] == "Count":
            value = np.random.randint(40, 150)
        elif char["type"] == "NumSecurities":
            # Use fixed value if known benchmark, else randomize
            if bench_code in FIXED_SECURITIES:
                value = FIXED_SECURITIES[bench_code]
            else:
                value = np.random.randint(20, 500)
        else:
            value = round(np.random.normal(loc=10, scale=2), 2)

        char_records.append({
            "BENCHMARK_CODE":         bench_code,
            "INCEPTION_YEAR":         inception_year,
            "CURRENCY_CODE":          currency_code,
            "CURRENCY":               currency_name,
            "CATEGORY":               "VC Benchmark",
            "CATEGORY_NAME":          "Venture Capital",
            "CHARACTERISTIC_NAME":    char["name"],
            "STATISTIC_TYPE":         char["type"],
            "UNIT":                   UNIT_MAP[char["type"]],
            "CHARACTERISTIC_VALUE":   value,
            "HISTORY_DATE":           TODAY
        })

df_benchmark_characteristics = pd.DataFrame(char_records)
print("\nBENCHMARK_CHARACTERISTICS")
display(HTML(df_benchmark_characteristics.to_html(index=False)))


BENCHMARK_CHARACTERISTICS


BENCHMARK_CODE,INCEPTION_YEAR,CURRENCY_CODE,CURRENCY,CATEGORY,CATEGORY_NAME,CHARACTERISTIC_NAME,STATISTIC_TYPE,UNIT,CHARACTERISTIC_VALUE,HISTORY_DATE
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Median IRR,Median,%,10.99,2025-08-03
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Mean MOIC,Mean,%,9.72,2025-08-03
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Top Quartile DPI,Percentile,%,11.30,2025-08-03
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Fund Count,Count,#,142.00,2025-08-03
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Std Dev IRR,StdDevIRR,%,13.05,2025-08-03
SP_500,2000,USD,US Dollar,VC Benchmark,Venture Capital,# of Securities,NumSecurities,#,500.00,2025-08-03
R2500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Median IRR,Median,%,8.17,2025-08-03
R2500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Mean MOIC,Mean,%,9.75,2025-08-03
R2500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Top Quartile DPI,Percentile,%,5.98,2025-08-03
R2500,2000,USD,US Dollar,VC Benchmark,Venture Capital,Fund Count,Count,#,143.00,2025-08-03


In [ ]:
# -- Snowflake SQL table creation

# CREATE TABLE BENCHMARK_CHARACTERISTICS (
#     BENCHMARK_CODE          VARCHAR(50)    NOT NULL,   -- Foreign key to BENCHMARK_GENERAL_INFORMATION
#     INCEPTION_YEAR          INTEGER        NOT NULL,   -- Typical inception year of the funds in the benchmark
#     CURRENCY_CODE           VARCHAR(5)     NOT NULL,   -- Currency code (e.g., USD, EUR)
#     CURRENCY                VARCHAR(40)    NOT NULL,   -- Full currency name
#     CATEGORY                VARCHAR(30)    NOT NULL,   -- e.g., 'VC Benchmark'
#     CATEGORY_NAME           VARCHAR(40)    NOT NULL,   -- e.g., 'Venture Capital'
#     CHARACTERISTIC_NAME     VARCHAR(50)    NOT NULL,   -- Statistic label (e.g., 'Median IRR', '# of Securities')
#     STATISTIC_TYPE          VARCHAR(30)    NOT NULL,   -- Type of statistic (e.g., 'Median', 'Count', 'NumSecurities')
#     UNIT                    VARCHAR(5)     NOT NULL,   -- %, #, etc.
#     CHARACTERISTIC_VALUE    NUMBER(18,4)   NOT NULL,   -- Simulated value (can be float or integer)
#     HISTORY_DATE            DATE           NOT NULL,   -- As-of date

#     CONSTRAINT FK_BENCHMARK_CHARACTERISTICS_BENCHMARK_CODE
#         FOREIGN KEY (BENCHMARK_CODE) REFERENCES BENCHMARK_GENERAL_INFORMATION(BENCHMARK_CODE)
# );

---

## **Table 3: BENCHMARK_PERFORMANCE**

The `BENCHMARK_PERFORMANCE` table records price index levels for each benchmark—**daily for public (market) benchmarks** and **quarterly for synthetic (VC/PE) strategies**.
This mixed frequency enables direct, realistic time-series comparison between venture strategies and public benchmarks in fact sheets, dashboards, and J-curve analysis.

Each row represents a single (date, benchmark) value—**daily for public benchmarks, quarterly for synthetic**.

### Generation Logic

| Step                           | Description                                                                                                                                                                                                                                                                                                                                                                                       |
| ------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. Benchmark Loop**          | For every benchmark in `BENCHMARK_GENERAL_INFORMATION`, determine its reporting currency and inception year using `BENCHMARK_CHARACTERISTICS`.                                                                                                                                                                                                                                                    |
| **2. Index Type Detection**    | If the benchmark is a public index (S&P 500, Russell 2000/2500, MSCI World), **daily closing levels are pulled using the [yfinance Python package](https://pypi.org/project/yfinance/)**, with the time series starting from the benchmark's inception year.                                                                                                                                    |
| **3. Synthetic VC Indices**    | For venture capital and private benchmarks, a synthetic "price index" (base=100) is generated **quarterly** from the inception year, reflecting realistic fund value trajectories:<br> - **Early Years:** Modest or flat growth<br> - **Growth Phase:** Accelerated mark-ups<br> - **Late Years:** Steady, slower compounding.                                 |
| **4. Placeholder for Future Daily Synthetic Logic** | For synthetic benchmarks, the code includes a clearly marked placeholder for future extension to daily frequency, should granular reporting ever be required. The quarterly logic can be swapped for daily by enabling a commented code block.                                                               |
| **5. Consistent Currency**     | Each row records both `CURRENCY_CODE` (ISO) and `CURRENCY` (full name), consistent with benchmark-level assignment for reporting.                                                                                                                                                                                                                           |
| **6. Time Window Enforcement** | For both real and synthetic indices, only periods from the benchmark's inception year through the most recent available date are reported, so series are directly comparable by inception.                                                                                                                                                                |
| **7. Validation**              | All `VALUE` fields are guaranteed floats (never arrays or lists). Rows with missing price data (package gaps) are excluded.                                                                                                                                                                                                                                |
| **8. Foreign Key**             | `BENCHMARK_CODE` maintains referential integrity to `BENCHMARK_GENERAL_INFORMATION` and downstream fact sheet components.                                                                                                                                                                                                                                   |

### Field Definitions

| Column                 | Description                                             | Example        |
|------------------------|---------------------------------------------------------|----------------|
| `BENCHMARK_CODE`       | Unique code for each benchmark                          | VC_US_GROWTH   |
| `PERFORMANCE_DATA_TYPE`| Type of measure (always 'PRICE' in this implementation) | PRICE          |
| `CURRENCY_CODE`        | ISO code (USD, EUR, etc.)                               | USD            |
| `CURRENCY`             | Full currency name                                      | US Dollar      |
| `PERFORMANCE_FREQUENCY`| **'Daily' for public benchmarks; 'Quarterly' for synthetic** | Daily / Quarterly |
| `VALUE`                | Index level (float)                                     | 125.43         |
| `HISTORY_DATE`         | Price date (YYYY-MM-DD); can be quarter-end or daily    | 2022-09-30     |

### Design Rationale and Alignment

- **Hybrid Data Source:** Public indices are actual market levels (e.g., S&P 500, MSCI World) pulled with the yfinance Python package at daily frequency; VC/private indices are simulated quarterly for realism.
- **Flexible Frequency:** Real indices support daily analysis and charting; synthetic benchmarks reflect true fund reporting practice (quarterly), but code is designed for future daily expansion if needed.
- **Aligned Inceptions:** Every benchmark’s time series starts at its own inception year for apples-to-apples analysis.
- **Consistent Index Construction:** All synthetic price series are generated using plausible VC logic—never negative in early years, no bracketed values, and appropriate compounding.
- **Schema Compliance:** Structure and keys align to Assette’s Snowflake data model and support downstream fact sheet automation.
- **Clean Output:** Output ready for reporting, charting, and IRR/J-curve calculations (to be performed downstream using the price series).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import yfinance as yf
from datetime import datetime
from IPython.display import display, HTML

random.seed(42)
np.random.seed(42)

# Mapping: BENCHMARK_CODE -> INCEPTION_YEAR
INCEPTION_MAP = df_benchmark_characteristics.drop_duplicates("BENCHMARK_CODE") \
    .set_index("BENCHMARK_CODE")["INCEPTION_YEAR"].to_dict()

# yfinance tickers for real indices
REAL_INDEX_MAP = {
    "SP_500":  "^GSPC",
    "R2500":   "IWM",    # IWM = Russell 2000 ETF
    "MSCI_WD": "URTH"    # URTH = MSCI World ETF
}

def get_daily_prices_yf(ticker, start, end):
    """Download daily close prices from yfinance and return as list of (price, date)."""
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty or "Close" not in df.columns:
        return []
    prices = df["Close"].round(2).values
    dates = [d.date() for d in df.index]
    return [(float(np.asarray(price).squeeze()), dt) for price, dt in zip(prices, dates)]

def get_quarterly_prices_yf(ticker, start, end):
    """(Retained for possible future use) Download quarterly prices if needed."""
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty or "Close" not in df.columns:
        return []
    df_q = df.resample('Q-DEC').last()
    if df_q.empty or "Close" not in df_q.columns:
        return []
    prices = df_q["Close"].round(2).values
    dates = [d.date() for d in df_q.index]
    return [(float(np.asarray(price).squeeze()), dt) for price, dt in zip(prices, dates)]

def simulate_vc_price_series(n, base=100):
    """Quarterly price simulation for synthetic VC/PE benchmarks."""
    prices = [base]
    for i in range(n):
        if i < 8:
            q_return = np.random.normal(0.005, 0.005)
        elif i < 20:
            q_return = np.random.normal(0.04, 0.01)
        else:
            q_return = np.random.normal(0.015, 0.007)
        prices.append(prices[-1] * (1 + q_return))
    return [round(x, 2) for x in prices[1:]]

# --- Placeholder for daily synthetic simulation (future enhancement) ---
# def simulate_vc_price_series_daily(n_days, base=100):
#     """Future enhancement: Simulate daily synthetic price series for VC/PE benchmarks."""
#     prices = [base]
#     for i in range(n_days):
#         # Insert realistic daily return logic for VC (tiny up/flat, random walk, etc.)
#         d_return = np.random.normal(0.00025, 0.001)  # placeholder for demonstration
#         prices.append(prices[-1] * (1 + d_return))
#     return [round(x, 2) for x in prices[1:]]

performance_records = []
today = datetime.today()
currency_code_default = "USD"
currency_name_default = "US Dollar"

for _, row in df_benchmark_general.iterrows():
    code = row["BENCHMARK_CODE"]
    currency_code = row.get("CURRENCY_CODE", currency_code_default)
    currency_name = row.get("CURRENCY", currency_name_default)
    performance_frequency = "Daily" if code in REAL_INDEX_MAP else "Quarterly"
    inception = INCEPTION_MAP.get(code, 2012)
    start_date = datetime(int(inception), 3, 31)  # First Q-end from inception year

    if code in REAL_INDEX_MAP:
        # Use daily prices from this inception year forward
        start = f"{inception}-01-01"
        end = today.strftime("%Y-%m-%d")
        yf_ticker = REAL_INDEX_MAP[code]
        d_data = get_daily_prices_yf(yf_ticker, start, end)
        for price, dt in d_data:
            if dt >= start_date.date():
                performance_records.append({
                    "BENCHMARK_CODE": code,
                    "PERFORMANCE_DATA_TYPE": "PRICE",
                    "CURRENCY_CODE": currency_code,
                    "CURRENCY": currency_name,
                    "PERFORMANCE_FREQUENCY": "Daily",
                    "VALUE": price,
                    "HISTORY_DATE": dt
                })
    else:
        # Quarterly simulation for synthetic benchmarks (VC/PE)
        n_quarters = min(48, (today.year - int(inception)) * 4 + today.month // 3)
        price_series = simulate_vc_price_series(n_quarters, base=100)
        for i in range(n_quarters):
            q_date = start_date + pd.DateOffset(months=3*i)
            performance_records.append({
                "BENCHMARK_CODE": code,
                "PERFORMANCE_DATA_TYPE": "PRICE",
                "CURRENCY_CODE": currency_code,
                "CURRENCY": currency_name,
                "PERFORMANCE_FREQUENCY": "Quarterly",
                "VALUE": price_series[i],
                "HISTORY_DATE": q_date.date()
            })
        # --- Placeholder for future daily synthetic logic ---
        # If daily synthetic simulation is needed, use the code below:
        # n_days = (today - start_date).days
        # price_series = simulate_vc_price_series_daily(n_days, base=100)
        # for i in range(n_days):
        #     d_date = start_date + pd.DateOffset(days=i)
        #     performance_records.append({
        #         "BENCHMARK_CODE": code,
        #         "PERFORMANCE_DATA_TYPE": "PRICE",
        #         "CURRENCY_CODE": currency_code,
        #         "CURRENCY": currency_name,
        #         "PERFORMANCE_FREQUENCY": "Daily",
        #         "VALUE": price_series[i],
        #         "HISTORY_DATE": d_date.date()
        #     })

df_benchmark_performance = pd.DataFrame(performance_records)[[
    "BENCHMARK_CODE",
    "PERFORMANCE_DATA_TYPE",
    "CURRENCY_CODE",
    "CURRENCY",
    "PERFORMANCE_FREQUENCY",
    "VALUE",
    "HISTORY_DATE"
]]

print("\nBENCHMARK_PERFORMANCE")
display(HTML(df_benchmark_performance.head(50).to_html(index=False)))


BENCHMARK_PERFORMANCE


BENCHMARK_CODE,PERFORMANCE_DATA_TYPE,CURRENCY_CODE,CURRENCY,PERFORMANCE_FREQUENCY,VALUE,HISTORY_DATE
SP_500,PRICE,USD,US Dollar,Daily,1498.58,2000-03-31
SP_500,PRICE,USD,US Dollar,Daily,1505.97,2000-04-03
SP_500,PRICE,USD,US Dollar,Daily,1494.73,2000-04-04
SP_500,PRICE,USD,US Dollar,Daily,1487.37,2000-04-05
SP_500,PRICE,USD,US Dollar,Daily,1501.34,2000-04-06
SP_500,PRICE,USD,US Dollar,Daily,1516.35,2000-04-07
SP_500,PRICE,USD,US Dollar,Daily,1504.46,2000-04-10
SP_500,PRICE,USD,US Dollar,Daily,1500.59,2000-04-11
SP_500,PRICE,USD,US Dollar,Daily,1467.17,2000-04-12
SP_500,PRICE,USD,US Dollar,Daily,1440.51,2000-04-13


In [ ]:
# -- Snowflake SQL table creation

# CREATE TABLE BENCHMARK_PERFORMANCE (
#     BENCHMARK_CODE            VARCHAR(50),     -- Foreign key to BENCHMARK_GENERAL_INFORMATION
#     PERFORMANCE_DATA_TYPE     VARCHAR(20),     -- Always 'PRICE' in this simulation (or could be 'IRR', 'MOIC', etc. if expanded)
#     CURRENCY_CODE             VARCHAR(10),     -- ISO code, e.g. 'USD'
#     CURRENCY                  VARCHAR(50),     -- Full currency name
#     PERFORMANCE_FREQUENCY     VARCHAR(30),     -- 'Daily' for public, 'Quarterly' for synthetic
#     VALUE                     NUMBER(18,2),    -- Index price level (never array)
#     HISTORY_DATE              DATE,            -- Date (quarter-end or daily)

#     FOREIGN KEY (BENCHMARK_CODE) REFERENCES BENCHMARK_GENERAL_INFORMATION(BENCHMARK_CODE)
# );

---

## **Table 4: BENCHMARK_ACCOUNT_ASSOCIATION**

The `BENCHMARK_ACCOUNT_ASSOCIATION` table defines the relationship between **accounts** and the **benchmarks** they are evaluated against. This association layer serves as the **join table** that connects account-level data with benchmark-level return and characteristic information.

Each row links an account (identified by `ACCOUNT_ID`) to one of its assigned benchmarks, with an ordinal `RANK` indicating priority.

---

### Generation Logic

- We use the list of account IDs from the **ACCOUNTS_MASTER** table (e.g., `ACC0001` through `ACC0050`).  
- Each account is randomly assigned **2 or 3 unique benchmarks** from the set of available benchmark codes.
- The `RANK` field indicates the priority: 1 (primary), 2 (secondary), 3 (tertiary), in assignment order.

---

| Field            | Description                                                    |
|------------------|---------------------------------------------------------------|
| `ACCOUNT_ID`     | Internal code for the account (FK to `ACCOUNTS_MASTER` table) |
| `BENCHMARK_CODE` | Reference to a benchmark (e.g., `PB_EU_HC`)                   |
| `RANK`           | Priority level: 1 (primary), 2 (secondary), 3 (tertiary)      |

---

**Notes:**
- This table enables flexible benchmarking: an account can be compared against multiple benchmarks, supporting composite benchmarking, relative performance reporting, and peer analysis.
- There are no duplicate (`ACCOUNT_ID`, `BENCHMARK_CODE`) pairs—each association is unique.

In [ ]:
import pandas as pd
import random
from IPython.display import display, HTML

# 1. Simulate 50 account IDs (ACC0001, ACC0002, ...)
NUM_ACCOUNTS = 50
ACCOUNT_IDS = [f"ACC{i+1:04}" for i in range(NUM_ACCOUNTS)]

BENCHMARK_CODES = df_benchmark_general["BENCHMARK_CODE"].tolist()

# 2. Build the association rows
assoc_rows = []
for account in ACCOUNT_IDS:
    # each account gets between 2 and 3 distinct benchmarks
    chosen = random.sample(BENCHMARK_CODES, k=random.randint(2, 3))
    for rank, bench in enumerate(chosen, start=1):
        assoc_rows.append({
            "ACCOUNT_ID":    account,
            "BENCHMARK_CODE": bench,
            "RANK":          rank
        })

# 3. Create the DataFrame
df_benchmark_account_association = pd.DataFrame(assoc_rows)

# 4. Validation: ensure no duplicates in account–benchmark pairing
if df_benchmark_account_association.duplicated(subset=["ACCOUNT_ID", "BENCHMARK_CODE"]).any():
    raise ValueError("Duplicate ACCOUNT_ID–BENCHMARK_CODE pairs found!")

# 5. Show all rows in the notebook output
pd.set_option('display.max_rows', None)

# Display the full association table
print("BENCHMARK_ACCOUNT_ASSOCIATION")
display(HTML(df_benchmark_account_association.to_html(index=False)))

BENCHMARK_ACCOUNT_ASSOCIATION


ACCOUNT_ID,BENCHMARK_CODE,RANK
ACC0001,SP_500,1
ACC0001,CA_NA,2
ACC0002,CA_US,1
ACC0002,MSCI_WD,2
ACC0003,PB_NA_GE,1
ACC0003,R2500,2
ACC0004,SP_500,1
ACC0004,CA_GE,2
ACC0004,R2500,3
ACC0005,CA_US,1


In [ ]:
# -- Snowflake SQL table creation

# CREATE TABLE BENCHMARK_ACCOUNT_ASSOCIATION (
#     ACCOUNT_ID      VARCHAR(20),                -- Account code (FK to ACCOUNTS_MASTER table, e.g., ACC0001)
#     BENCHMARK_CODE  VARCHAR(50),                -- Linked benchmark (FK to BENCHMARK_GENERAL_INFORMATION)
#     RANK            NUMBER(1),                  -- Importance: 1=primary, 2=secondary, 3=tertiary

#     PRIMARY KEY (ACCOUNT_ID, BENCHMARK_CODE),
#     FOREIGN KEY (ACCOUNT_ID) REFERENCES ACCOUNTS_MASTER(ACCOUNT_ID),
#     FOREIGN KEY (BENCHMARK_CODE) REFERENCES BENCHMARK_GENERAL_INFORMATION(BENCHMARK_CODE)
# );

---